In [4]:
import os
import json
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 修正为 langchain.text_splitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOpenAI

# 检查CUDA是否可用
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 设置OpenAI API密钥和自定义API_BASE
api_key = "sk-PzbRqOZZQct2jv7f5LTtTH0D19GtPKfsey74QqPkthOtJBYh"  # 你的API密钥
api_base = "https://xdaicn.top/v1"


In [14]:
from rag_for_longchain.utils.preprocessing import JSONChunkSplitter
directory = r"D:\converstional_rag\23acldata\matches"
max_words = 1024

# 创建类实例
splitter = JSONChunkSplitter(directory, max_words)

# 调用 process_directory 处理整个目录下的所有 JSON 文件
all_chunks = splitter.process_directory()
all_chunks = [chunk for chunk in all_chunks if chunk.get("stance") != "MIXED"]

print(f"切分后的 chunk 总数: {len(all_chunks)}")

正在处理文件: 01悉尼大学VS南京航空航天大学_应该_不应该鼓励00后整顿职场.json
正在处理文件: 02大连理工大学VS复旦大学_应该_不应该鼓励00后整顿职场.json
正在处理文件: 03湖南财政经济学院VS井冈山大学_“东方甄选”模式未来可期_难以长久.json
正在处理文件: 04东北林业大学VS厦门大学嘉庚学院_“东方甄选”模式未来可期_难以长久.json
正在处理文件: 05帝国理工大学VS天津大学_“顶流”出演话剧振兴_扰乱了话剧市场.json
正在处理文件: 06厦门大学VS清华大学_“顶流”出演话剧振兴_扰乱了话剧市场.json
正在处理文件: 07广东外语外贸大学VS江西财经大学_当下大学生因为就业难而选择考研是_不是一个明智的选择.json
正在处理文件: 08西南财经大学VS纽约大学_当下大学生因为就业难而选择考研是_不是一个明智的选择.json
正在处理文件: 09南京航空航天大学VS北京大学_应该_不应该鼓励00后整顿职场.json
正在处理文件: 10复旦大学VS武汉大学_应该_不应该鼓励00后整顿职场.json
正在处理文件: 11井冈山大学VS太原科技大学_“东方甄选”模式未来可期_难以长久.json
正在处理文件: 12厦门大学嘉庚学院VS国际关系学院_“东方甄选”模式未来可期_难以长久.json
正在处理文件: 12进6淘汰赛AB_科技是自由意志的牢笼_科技是自由意志的羽翼.json
正在处理文件: 12进6淘汰赛CD_低欲望社会的形成是社会进步_退步的表现.json
正在处理文件: 12进6淘汰赛EF_应该_不应该以侵权为由遏止短视频二创的疯狂增长.json
正在处理文件: 12进6淘汰赛GH_当代文学作品在翻译时，应该_不应该支持译者的“创造”.json
正在处理文件: 12进6淘汰赛IJ_后疫情时代加速_减缓了全球化趋势.json
正在处理文件: 12进6淘汰赛KL_选举后，民选议员跳槽应该_不应该接受法律制裁.json
正在处理文件: 13天津大学VS中国海洋大学_“顶流”出演话剧振兴_扰乱了话剧市场.json
正在处理文件: 14清华大学VS海南大学_“顶流”出演话剧振兴_扰乱了话剧市场.json
正在处理文件: 15江西财经大学VS澳门科技大学_当下大学生因为就业难而选择考研是_不是一个明智的选择.json


In [16]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import json
import os
import torch

# 模型和存储配置
model_name = "hfl/chinese-roberta-wwm-ext-large"  # 使用 hfl/chinese-roberta-wwm-ext-large 模型
output_dir = "D:/converstional_rag/rag_for_longchain/experiments"  # 嵌入向量存储路径

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 确保存储路径存在
os.makedirs(output_dir, exist_ok=True)

# 嵌入生成函数
def generate_embeddings(data, tokenizer, model):
    """
    为每条数据生成嵌入向量。
    :param data: 包含 stance, debater, utterance 等信息的字典列表
    :param tokenizer: 分词器
    :param model: 预训练模型
    :return: 嵌入向量列表
    """
    embeddings = []
    for item in data:
        chunk = item['utterance']  # 提取辩论文本内容
        # 对每个 utterance 进行 tokenization 和向量化
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # 使用输出的最后一层隐藏状态的均值作为嵌入
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(embedding)
    return np.array(embeddings)

# 示例数据：包含 stance、debater、utterance 和 labels
data = all_chunks

# 生成嵌入向量
embeddings = generate_embeddings(data, tokenizer, model)

# 提取原始信息
stances = [item["stance"] for item in data]
debaters = [item["debater"] for item in data]
utterances = [item["utterance"] for item in data]


# 保存嵌入和上下文信息到结构化文件
embedding_file = os.path.join(output_dir, "structured_embeddings_for_naive_rag.npz")
np.savez(
    embedding_file,
    embeddings=embeddings,
    stances=stances,
    debaters=debaters,
    utterances=utterances,
)

print(f"嵌入生成完成，存储在: {embedding_file}")

嵌入生成完成，存储在: D:/converstional_rag/rag_for_longchain/experiments\structured_embeddings_for_naive_rag.npz


In [ ]:
import os
import json
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch

from rag_for_longchain.retriever.faiss_retriever import retrieve_candidates
from rag_for_longchain.generator.testgen import generate_counterargument_via_api

# -------------------- 模型与分词器初始化 --------------------
MODEL_NAME = "hfl/chinese-roberta-wwm-ext-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# -------------------- 重用辅助函数 --------------------
def extract_utterances(data):
    utterances = []
    if isinstance(data, str):
        for line in data.splitlines():
            line = line.strip()
            if line:
                utterances.append(line)
    elif isinstance(data, list):
        for item in data:
            utterances.extend(extract_utterances(item))
    elif isinstance(data, dict):
        for v in data.values():
            utterances.extend(extract_utterances(v))
    else:
        try:
            utterances.extend(extract_utterances(str(data)))
        except:
            pass
    return utterances

def recursive_split(text, max_length=512):
    tokens = tokenizer.tokenize(text)
    if len(tokens) <= max_length:
        return [text]
    mid = len(tokens) // 2
    part1 = tokenizer.convert_tokens_to_string(tokens[:mid])
    part2 = tokenizer.convert_tokens_to_string(tokens[mid:])
    return recursive_split(part1, max_length) + recursive_split(part2, max_length)

def generate_embeddings(chunks):
    embs = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        embs.append(emb)
    return np.vstack(embs)

def process_chunks(chunks):
    all_texts = []
    skill_scores = {}
    for c in chunks:
        all_texts.append(c["text"])
        labels = c.get("labels", {})
        for skill, info in labels.items():
            try:
                score = float(info.get("评分", 0))
            except:
                score = 0
            skill_scores[skill] = skill_scores.get(skill, 0) + score
    if skill_scores:
        best_skill = max(skill_scores, key=skill_scores.get)
        best_score = skill_scores[best_skill]
    else:
        best_skill, best_score = None, 0
    return {
        "所有文本": all_texts,
        "最佳辩论技巧": best_skill,
        "最高评分": best_score
    }
